In [1]:
# !pip install vncorenlp
# !git clone https://github.com/vncorenlp/VnCoreNLP.git

In [2]:
# !wget https://github.com/vncorenlp/VnCoreNLP/archive/refs/tags/v1.2.zip
# !unzip v1.2.zip
!pip install underthesea


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.1 MB/s eta 0:00:00


In [3]:
from underthesea import word_tokenize
import os

# Hàm tách từ sử dụng Underthesea
def word_segment(text):
    return word_tokenize(text, format='text')



In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

data = pd.read_csv('/kaggle/input/tk2-deepleaning/df_final (1).csv')
data = data.dropna().reset_index()
data['cleaned_segmented'] = data['cleaned_old_data'].progress_apply(word_segment)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_segmented'], data['label'], test_size=0.2, random_state=42)

# Chuyển đổi văn bản thành vector TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)



  0%|          | 0/71121 [00:00<?, ?it/s]

In [5]:
# Huấn luyện mô hình Naive Bayes
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test_tfidf)

# Đánh giá mô hình
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.71      0.70      2380
           1       0.70      0.75      0.72      2315
           2       0.64      0.58      0.61      2415
           3       0.79      0.78      0.79      2416
           4       0.77      0.71      0.74      2257
           5       0.82      0.90      0.86      2442

    accuracy                           0.74     14225
   macro avg       0.74      0.74      0.74     14225
weighted avg       0.74      0.74      0.74     14225



In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

clf = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
clf.fit(X_train_tfidf, y_train)

# Dự đoán và đánh giá mô hình
y_pred = clf.predict(X_test_tfidf)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.7663    0.7689    0.7676      2380
           1     0.7381    0.8009    0.7682      2315
           2     0.7292    0.6567    0.6911      2415
           3     0.8625    0.8853    0.8738      2416
           4     0.8521    0.8197    0.8356      2257
           5     0.9368    0.9586    0.9476      2442

    accuracy                         0.8155     14225
   macro avg     0.8142    0.8150    0.8140     14225
weighted avg     0.8146    0.8155    0.8144     14225



In [7]:
# pip install xgboost

import xgboost as xgb

# Chuyển đổi nhãn thành mảng numpy
y_train_np = y_train.values
y_test_np = y_test.values

# Tạo DMatrix cho XGBoost
dtrain = xgb.DMatrix(X_train_tfidf, label=y_train_np)
dtest = xgb.DMatrix(X_test_tfidf, label=y_test_np)

# Thiết lập tham số cho XGBoost
params = {
    'objective': 'multi:softmax',
    'num_class': 6,
    'eval_metric': 'merror',
    'max_depth': 6,
    'eta': 0.1,
    'seed': 42
}

# Huấn luyện mô hình
bst = xgb.train(params, dtrain, num_boost_round=100)

# Dự đoán
y_pred = bst.predict(dtest)

# Đánh giá mô hình
print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

           0     0.7519    0.7706    0.7612      2380
           1     0.6770    0.7516    0.7124      2315
           2     0.7285    0.6600    0.6926      2415
           3     0.8923    0.8609    0.8763      2416
           4     0.8321    0.8170    0.8245      2257
           5     0.9492    0.9644    0.9567      2442

    accuracy                         0.8047     14225
   macro avg     0.8052    0.8041    0.8040     14225
weighted avg     0.8062    0.8047    0.8048     14225



In [8]:
# Cài đặt các thư viện cần thiết
# pip install transformers torch scikit-learn pandas

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torch.nn.utils import clip_grad_norm_

# Thiết lập thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

# Tải tokenizer và model PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=6)
model.to(device)

# Tiền xử lý dữ liệu cho PhoBERT
def encode_texts(texts, tokenizer, max_length=256):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

print("Bắt đầu mã hóa dữ liệu...")
X_train_enc = encode_texts(X_train, tokenizer)
X_test_enc = encode_texts(X_test, tokenizer)
print("Hoàn tất mã hóa dữ liệu.")

# Chuyển đổi nhãn thành tensor
y_train_tensor = torch.tensor(y_train.values)
y_test_tensor = torch.tensor(y_test.values)

# Tạo DataLoader
batch_size = 32  # Giảm batch size để tránh vấn đề về bộ nhớ GPU

train_dataset = TensorDataset(X_train_enc['input_ids'], X_train_enc['attention_mask'], y_train_tensor)
test_dataset = TensorDataset(X_test_enc['input_ids'], X_test_enc['attention_mask'], y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Huấn luyện mô hình
optimizer = AdamW(model.parameters(), lr=1e-5)  # Giảm learning rate từ 2e-5 xuống 1e-5

num_epochs = 10  # Tăng số epoch để mô hình có thời gian học nhiều hơn
total_steps = len(train_loader) * num_epochs

# Thiết lập scheduler để điều chỉnh learning rate
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Hàm tính độ chính xác
def compute_accuracy(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(preds_flat == labels_flat) / len(labels_flat)

# Early Stopping parameters
patience = 2
best_accuracy = 0
epochs_no_improve = 0

print("Bắt đầu huấn luyện mô hình...")
for epoch in range(num_epochs):
    print(f"\n===== Epoch {epoch + 1} / {num_epochs} =====")
    model.train()
    total_loss = 0
    total_accuracy = 0

    for step, batch in enumerate(train_loader):
        model.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        # Tính độ chính xác trên batch hiện tại
        logits = logits.detach().cpu().numpy()
        label_ids = labels.cpu().numpy()
        batch_accuracy = compute_accuracy(logits, label_ids)
        total_accuracy += batch_accuracy

        loss.backward()
        # Clip gradients để tránh gradient explosion
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        if step % 10 == 0 and step != 0:
            avg_loss = total_loss / (step + 1)
            avg_accuracy = total_accuracy / (step + 1)
            print(f"  Batch {step}/{len(train_loader)} - Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}")

    avg_epoch_loss = total_loss / len(train_loader)
    avg_epoch_accuracy = total_accuracy / len(train_loader)
    print(f"\nKết quả Epoch {epoch + 1}:")
    print(f"  Loss trung bình: {avg_epoch_loss:.4f}")
    print(f"  Độ chính xác trung bình: {avg_epoch_accuracy:.4f}")

    # Đánh giá và kiểm tra Early Stopping
    if avg_epoch_accuracy > best_accuracy:
        best_accuracy = avg_epoch_accuracy
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pt')
        print("  -> Cập nhật mô hình tốt nhất.")
    else:
        epochs_no_improve += 1
        print(f"  -> Không cải thiện được trong {epochs_no_improve} epoch.")
        if epochs_no_improve >= patience:
            print("Early stopping được kích hoạt.")
            break

print("Hoàn tất huấn luyện.")

# Lưu mô hình và tokenizer sử dụng save_pretrained
save_directory = 'saved_phobert_model'

# Tạo thư mục lưu nếu chưa tồn tại
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Lưu mô hình
model.save_pretrained(save_directory)

# Lưu tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Mô hình và tokenizer đã được lưu tại {save_directory}")

# Đánh giá mô hình trên tập kiểm tra
print("\nBắt đầu đánh giá mô hình trên tập kiểm tra...")
model.load_state_dict(torch.load('/kaggle/working/best_model.pt'))  # Tải mô hình tốt nhất
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        y_pred.extend(predictions.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

print("Hoàn tất đánh giá.")

print("\nBáo cáo đánh giá:")
print(classification_report(y_true, y_pred, digits=5))


Sử dụng thiết bị: cuda


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

...

Bắt đầu mã hóa dữ liệu...
Hoàn tất mã hóa dữ liệu.


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Bắt đầu huấn luyện mô hình...

===== Epoch 1 / 10 =====
  Batch 10/1778 - Loss: 1.7850, Accuracy: 0.1534
  Batch 20/1778 - Loss: 1.7771, Accuracy: 0.1845
  Batch 30/1778 - Loss: 1.7583, Accuracy: 0.2349
  Batch 40/1778 - Loss: 1.7462, Accuracy: 0.2668
  Batch 50/1778 - Loss: 1.7327, Accuracy: 0.2862
  Batch 60/1778 - Loss: 1.7137, Accuracy: 0.3135
  Batch 70/1778 - Loss: 1.6975, Accuracy: 0.3310
  Batch 80/1778 - Loss: 1.6840, Accuracy: 0.3445
  Batch 90/1778 - Loss: 1.6690, Accuracy: 0.3520
  Batch 100/1778 - Loss: 1.6499, Accuracy: 0.3648
  Batch 110/1778 - Loss: 1.6363, Accuracy: 0.3725
  Batch 120/1778 - Loss: 1.6185, Accuracy: 0.3851
  Batch 130/1778 - Loss: 1.5996, Accuracy: 0.3972
  Batch 140/1778 - Loss: 1.5856, Accuracy: 0.4058
  Batch 150/1778 - Loss: 1.5707, Accuracy: 0.4131
  Batch 160/1778 - Loss: 1.5568, Accuracy: 0.4175
  Batch 170/1778 - Loss: 1.5416, Accuracy: 0.4234
  Batch 180/1778 - Loss: 1.5294, Accuracy: 0.4273
  Batch 190/1778 - Loss: 1.5123, Accuracy: 0.4352
  B

/tmp/ipykernel_24/1186051907.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/best_model.pt'))  # Tải mô hình tốt nhất

Hoàn tất đánh giá.

Báo cáo đánh giá:
              precision    recall  f1-score   support

           0    0.85252   0.83067   0.84146      2380
           1    0.79335   0.82419   0.80847      2315
           2    0.80035   0.75197   0.77541      2415
           3    0.93165   0.96482   0.94795      2416
           4    0.88423   0.89677   0.89045      2257
           5    0.98320   0.98280   0.98300      2442

    accuracy                        0.87564     14225
   macro avg    0.87422   0.87520   0.87446     14225
weighted avg    0.87494   0.87564   0.87503     14225

